In [ ]:
import torch
import whisper

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
print(torch.__version__)

model = whisper.load_model("medium", device=device)
result = model.transcribe("sample_convo.wav")
transcript = result["text"]
print("Transcript:", transcript)


Device: cuda
2.6.0+cu126
Transcript: Okay. Yeah. Good morning. Good morning. Nice to see you. Yeah. Oh, I'm sorry. Thank you. I need it opened. I don't have strength to open it. I guess so. Yeah, I want, sorry, I need my vitals and my temperature check because I'm not sure if I still have a fever or not. Do you want to call your mom? She might. Let me see. We can FaceTime with her, just like yesterday? Yeah. That last you like? Yeah. That last like this time? Mm-hmm. Oh, no, no, no, sorry, the right one. Here we go. Can you hear me? Hello? Can you hear me? Can you hear me? I don't know what's wrong with this. It's, sorry, it's something wrong with the Wi-Fi. You want me to find my phone? Let me just try one more time. I'll turn the Wi-Fi on. Oh, do you mind taking this one too? I got it. They're cold. Oh, it's right here. They don't stay hot very long. Can you hear me now? You can hear me, but I can't hear you. Okay. This is my auntie. Okay. She's on the phone with my mom. I'm not sure

In [ ]:
from google import genai

def generate_medical_summary(transcript):
    client = genai.Client(api_key="YOUR_API_KEY")

    prompt = f"""
    You are an AI assistant helping doctors summarize patient consultations. 
    Given the following doctor-patient conversation, extract key medical details in bullet points:
    
    --- Transcript ---
    {transcript}
    -----------------
    
    Provide a structured summary in the following format:
    - Chief Complaint: [Main issue]
    - Symptoms: [List of symptoms]
    - Possible Diagnosis: [Doctor's suspected condition]
    - Family History: [Relevant family conditions]
    - Prescription: [Medications and recommendations]
    - Follow-up Instructions: [Next steps]
    - Signatures: [Doctor's name and credentials]
    """

    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt],
    )


    return response.text

final_summary = generate_medical_summary(transcript)

print(final_summary)


- Chief Complaint: Severe headaches for the past few days.
- Symptoms: Nausea, dizziness.
- Possible Diagnosis: Stress-induced headaches or migraines.
- Family History: Mother had frequent migraines.
- Prescription: Paracetamol and rest.
- Follow-up Instructions: None mentioned.



In [42]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_prescription_pdf(summary, prescription, output_file):
    c = canvas.Canvas(output_file, pagesize=letter)
    width, height = letter

    # Header
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, height - 50, "Medical Prescription Summary")
    
    # Add the summary of key points
    c.setFont("Helvetica", 12)
    text_object = c.beginText(50, height - 80)
    text_object.textLine("Key Points from the Consultation:")
    for line in summary.splitlines():
        text_object.textLine(line)
    c.drawText(text_object)
    
    # Add a gap
    c.line(50, height - 200, width - 50, height - 200)
    
    # Signature line for the doctor
    c.line(50, 100, 250, 100)
    c.drawString(50, 80, "Doctor's Signature:")
    
    c.save()

# Create the PDF using the extracted summary and prescription text
create_prescription_pdf(final_summary, prescription_text, "prescription.pdf")